<a href="https://colab.research.google.com/github/theBuluku/Independent-Project-week-9/blob/master/IP_Week_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [0]:
#Loading the train dataset(dataset1) and viewing the first 5 rows.
train_df=pd.read_csv('train.csv')
train_df.head(6)

In [14]:
#Showing the rows and columns.
train_df.shape

(891, 12)

In [0]:
#Loading the test dataset(dataset2) and viewing the first 5 rows.
test_df=pd.read_csv('test.csv')
test_df.head(6)

In [16]:
#Showing the rows and columns.
test_df.shape

(418, 11)

In [0]:
#Loading the spambase dataset(dataset3) and viewing the first 5 rows.
spambase_df=pd.read_csv('spambase.data')
spambase_df.head(6)

In [19]:
#Showing the rows and columns.
spambase_df.shape

(4600, 58)

#Exploratory Data Analysis.

In [11]:
# Previewing the information of the dataset
#
print("train dataset infomation")
print("********************")
print(train_df.info())
print("test dataset infomation")
print("********************")
print(test_df.info())

train dataset infomation
********************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB
None
test dataset infomation
********************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch 

In [12]:
# Previewing the statistical information of the dataset
#
print("train dataset stastical information")
print("********************")
print(train_df.describe())
print("test dataset stastical information")
print("********************")
print(test_df.describe())

train dataset stastical information
********************
       PassengerId    Survived      Pclass  ...       SibSp       Parch        Fare
count   891.000000  891.000000  891.000000  ...  891.000000  891.000000  891.000000
mean    446.000000    0.383838    2.308642  ...    0.523008    0.381594   32.204208
std     257.353842    0.486592    0.836071  ...    1.102743    0.806057   49.693429
min       1.000000    0.000000    1.000000  ...    0.000000    0.000000    0.000000
25%     223.500000    0.000000    2.000000  ...    0.000000    0.000000    7.910400
50%     446.000000    0.000000    3.000000  ...    0.000000    0.000000   14.454200
75%     668.500000    1.000000    3.000000  ...    1.000000    0.000000   31.000000
max     891.000000    1.000000    3.000000  ...    8.000000    6.000000  512.329200

[8 rows x 7 columns]
test dataset stastical information
********************
       PassengerId      Pclass         Age       SibSp       Parch        Fare
count   418.000000  418.000000

In [13]:
# Getting the names of the columns

print("names of the columns of the train dataset ")
print("********************")
print(train_df.columns)
print("names of the columns of the test dataset ")
print("********************")
print(test_df.columns)

names of the columns of the train dataset 
********************
Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
names of the columns of the test dataset 
********************
Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


#SPLITTING.

#1. Splitting train data.

In [0]:
#Splitting the dataset into train and test
train_data, test_data = train_test_split(train_df, test_size=0.2)

In [24]:
#Showing the rows and columns of train dataset
train_data.shape

(712, 12)

In [25]:
test_data.shape

(179, 12)

#2. Splitting test data

In [0]:
#Splitting the dataset into train and test
train_data1, test_data1 = train_test_split(test_df, test_size=0.2)

In [31]:
train_data1.shape

(334, 11)

In [32]:
test_data1.shape

(84, 11)

#3. Splitting test data

In [0]:
#Splitting the dataset into train and test
train_data2, test_data2 = train_test_split(spambase_df, test_size=0.2)

In [35]:
train_data2.shape

(3680, 58)

In [36]:
test_data2.shape

(920, 58)